In [1]:
using LinearAlgebra, SparseArrays
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [2]:
gmsh.open("4node1st.geo")

Info    : Reading '4node1st.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Line)
Info    : Done meshing 1D (Wall 0.000282885s, CPU 0.000284s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 5.2031e-05s, CPU 5.2e-05s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 60%] Meshing curve 3 order 2
Info    : [ 80%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 1
Info    : Done meshing order 2 (Wall 5.0011e-05s, CPU 5e-05s)
Info    : 6 nodes 7 elements
Info    : Done reading '4node1st.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type="PlaneStress")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 5)


LowLevelFEM.Problem("4node1st", "PlaneStress", 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 6)

In [4]:
gmsh.model.mesh.getElementsByCoordinates(0, 0.5, 0, -1)

1-element Vector{UInt64}:
 0x0000000000000007

In [5]:
gmsh.model.mesh.getLocalCoordinatesInElement(7, -0.5, 0, 0)

(0.0, 0.0, 0.0)

In [6]:
gmsh.model.mesh.getLocalCoordinatesInElement(7, 0.5, 0, 0)

(1.0, 0.0, 0.0)

In [7]:
gmsh.model.mesh.getLocalCoordinatesInElement(7, 0, 0.866, 0)

(1.0486862840177719e-16, 0.9999999999999999, 0.0)

In [8]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [9]:
K = FEM.stiffnessMatrix(problem)

12×12 SparseMatrixCSC{Float64, Int64} with 144 stored entries:
      1.06268e5    35714.3         …  -29608.6         -43956.0
  35714.3          65031.2            -51282.1         -84595.9
  28020.5           -915.751               1.26985e-7       6.92226e-8
    915.751          528.073               6.92266e-8      -1.1978e-7
   7402.14         12820.5            -29608.6         -51282.1
  10989.0          21149.0         …  -43956.0         -84595.9
     -1.12082e5     3663.0            -59217.1          95238.1
  -3663.0          -2112.29            95238.1             -1.69192e5
     -1.29179e-7       7.34544e-8         -2.24164e5        2.00159e-7
      7.34553e-8       1.27186e-7          2.00148e-7   -4224.59
 -29608.6         -51282.1         …       3.42599e5        1.70928e-7
 -43956.0         -84595.9                 1.70928e-7       3.42608e5

In [10]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
load = FEM.load("load", fy=-1)

("load", 0, -1, 0)

In [11]:
f = FEM.loadVector(problem, [load])

Error   : Physical name 'load' does not exist


ErrorException: Physical name 'load' does not exist

In [12]:
FEM.applyBoundaryConditions!(problem, K, f, [supp])

UndefVarError: UndefVarError: `f` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
q = FEM.solveDisplacement(K, f)

UndefVarError: UndefVarError: `f` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [14]:
q = [0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0]

12-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.5
 0.0
 0.0
 0.0
 0.0
 0.0

In [15]:
u0 = FEM.showDoFResults(problem, q, "ux", name="shape1", visible=false)

1

In [16]:
S = FEM.solveStress(problem, q)

LowLevelFEM.TensorField([[439560.43956274784; -88825.72393011648; … ; 0.0; 0.0;;]], [7], 1, "s")

In [17]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [18]:
u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)
Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


12

In [19]:
FEM.plotOnPath(problem, "path", sx, 100, name="σx", visible=false);
FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 100, name="ux", visible=false);

MethodError: MethodError: no method matching plotOnPath(::LowLevelFEM.Problem, ::String, ::Int32, ::Int64; name::String, visible::Bool)
The function `plotOnPath` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  plotOnPath(::Any, ::Any, ::Any; points, step, plot, name, visible)
   @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/LowLevelFEM.jl:4052


In [20]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()